# Test the fortran functions

In [ ]:
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import diskwarp
%matplotlib widget

## 1. Test seach for intersections

Define a point

In [ ]:
p = [.3,.5]

Define some 3D surface mesh (add noise to potentiall have several surfaces overlap)

In [ ]:
_x = np.sin(np.linspace(0,np.pi,15))
_y = np.linspace(0, 1, 10)

xi, yi = np.meshgrid(_x, _y, indexing='ij')
xi += 0.3 * np.random.rand(*xi.shape)
yi += 0.3 * np.random.rand(*yi.shape)
zi = np.linspace(0, 1, len(_x))[:,None] * np.ones_like(xi)

vi = 5. * np.ones_like(xi)

Now search for the surfaces that lie along the projection in z-direction

In [ ]:
matches, n_matches = diskwarp.fmodule.findpoint(xi, yi, p)
matches -= 1
print(f'{n_matches} cells found')

Now search for the intersection points and interpolate

In [ ]:
intersect, n_inter = diskwarp.fmodule.intersect_surface(xi, yi, zi, vi, p, 10)
intersect = intersect[:n_inter]

In [ ]:
fig = plt.figure(figsize=(8, 4))

ax = fig.add_axes([0, 0, 0.5, 1], projection='3d')

# plot 3d mesh

for ix in range(xi.shape[0]):
    ax.plot(xi[ix, :], yi[ix, :], zi[ix, :], lw=0.5, c='k');
for iy in range(xi.shape[1]):
    ax.plot(xi[:, iy], yi[:, iy], zi[:, iy], lw=0.5, c='k');
    
# plot matches in 3d
    
for i in range(n_matches):
    ix, iy = matches[i, :]
    ax.plot(
        [xi[ix,iy], xi[ix+1,iy], xi[ix+1,iy+1], xi[ix,iy+1], xi[ix,iy]],
        [yi[ix,iy], yi[ix+1,iy], yi[ix+1,iy+1], yi[ix,iy+1], yi[ix,iy]],
        [zi[ix,iy], zi[ix+1,iy], zi[ix+1,iy+1], zi[ix,iy+1], zi[ix,iy]],
        )
    
for i in range(n_inter):
    ax.scatter(*intersect[i,:3], 'o')
    
# plot the "ray"

ax.plot([p[0], p[0]], [p[1],p[1]], [0, 1], lw=1, c='r');
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

# now the same in 2D

ax = fig.add_axes([0.5, 0, 0.5, 1])

# plot the mesh
ax.plot(xi, yi, c='k')
ax.plot(xi.T, yi.T, c='k')

# plot the point
ax.plot(*p, 'ro')

# plot the squares
for i in range(n_matches):
    ix, iy = matches[i, :]
    plt.plot(
        [xi[ix,iy], xi[ix+1,iy], xi[ix+1,iy+1], xi[ix,iy+1], xi[ix,iy]],
        [yi[ix,iy], yi[ix+1,iy], yi[ix+1,iy+1], yi[ix,iy+1], yi[ix,iy]],
        )

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

## 2. Test bilinear interpolation

Define 4 points with their values

In [ ]:
p00 = [1.0, 1.0, 1.0]
p10 = [2.0, 0.0, 2.0]
p11 = [3.5, 2.0, 2.0]
p01 = [1.5, 3.0, 1.0]

v00 = 1.
v10 = 2.
v11 = 3.
v01 = 2.

convert them to a 2x2 grid

In [ ]:
xi, yi, zi = np.array([
    [p00, p01],
    [p10, p11]]).T

vi = np.array([
    [v00, v01],
    [v10, v11]]).T

Make the regular image grid

In [ ]:
_x = np.linspace(0, 4, 20)
_y = np.linspace(0, 4, 30)
img_x, img_y = np.meshgrid(_x, _y, indexing='ij')
img_xc = 0.5 * (img_x[1:, 1:] + img_x[:-1, 1:])
img_yc = 0.5 * (img_y[1:, 1:] + img_y[1:, :-1])
img_zc = np.ones_like(img_xc)

Interpolate each pixels on the grid

In [ ]:
img_z, img_v = diskwarp.fmodule.interpolate_grid(xi, yi, zi, vi, img_xc, img_yc)

In [ ]:
c = [v00, v10, v11, v01, v00]
col = plt.cm.viridis(Normalize(vmin=1, vmax=3)(c))

In [ ]:
points = np.array([xi.ravel(), yi.ravel()]).T
values = vi.ravel()
new_points = np.array([img_xc.ravel(), img_yc.ravel()]).T
new_values = griddata(points, values, new_points).reshape(img_xc.shape)

In [ ]:
mask = ~np.isnan(new_values)
np.allclose(img_v[mask], new_values[mask])

Read the fortran created test data

In [ ]:
f, axs = plt.subplots(1, 2, dpi=100, sharex=True, sharey=True, figsize=(10, 4))
for ax in axs:
    ax.set_aspect('equal')

cc0 = axs[0].pcolormesh(img_x, img_y, new_values, vmin=1, vmax=3)
plt.colorbar(cc0, ax=axs[0])
cc1 = axs[1].pcolormesh(img_x,    img_y,    img_v,         vmin=1, vmax=3)
plt.colorbar(cc1, ax=axs[1])

for ax in axs:
    ax.plot(img_x, img_y, 'k', lw=0.1)
    ax.plot(img_x.T, img_y.T, 'k', lw=0.1)
    ax.plot(img_xc, img_yc, 'ok', ms=1)
    ax.plot(*np.array([p00, p10, p11, p01, p00]).T[:2])
    ax.scatter(*np.array([p00, p10, p11, p01, p00]).T[:2], c=c, ec='w', s=55);

## 3. Test `pointintriangle`

In [ ]:
pt = [2., 1.]

This should be `True`

In [ ]:
bool(diskwarp.fmodule.pointintriangle(pt, p00[:2], p10[:2], p11[:2]))

This should be `False`

In [ ]:
bool(diskwarp.fmodule.pointintriangle(pt, p00[:2], p11[:2], p01[:2]))

In [ ]:
f, ax = plt.subplots()
ax.plot(*np.array([p00, p10, p11, p00]).T[:2]);
ax.plot(*np.array([p00, p11, p01, p00]).T[:2]);
ax.scatter(pt[0], pt[1], c='r', marker='o')
ax.scatter(*np.array([p00, p10, p11, p01, p00]).T[:2], c=c, ec='w', s=50);

## 4. Test `intersect_triangle`

In [ ]:
A  = np.array([2.0, 0.0, 1.0])
B  = np.array([3.0, 2.0, 2.0])
C  = np.array([1.0, 1.5, 3.0])

VA, VB, VC = 1., 2., 3.

P = np.array([2.0, 1.0, 1.5])

Determine intersection in python and fortran

In [ ]:
def intersect_tri(A, B, C, P):
    n = np.cross((B - A), (C - A))
    n = n / np.linalg.norm(n)
    d_plane = n.dot(A)
    ray = np.array([0, 0, 1])
    t = ( d_plane - n.dot(P)) / (n.dot(ray))
    Q = P + t * ray
    return Q

Qp = intersect_tri(A, B, C, P)
Qf = diskwarp.fmodule.intersect_triangle(A, B, C, P)

Check if we are identical (should return `True`)

In [ ]:
np.allclose(Qp, Qf)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot(*np.array([A, B, C, A]).T)
ax.plot(*np.array([P, Qf]).T, '-o')

Randomly place points in the triangle and color according to interpolation

In [ ]:
vmin = np.array([A, B, C]).min(0)
vmax = np.array([A, B, C]).max(0)

In [ ]:
norm = Normalize(vmin=1, vmax=3)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot(*np.array([A, B, C, A]).T)
ax.scatter(*np.array([A, B, C, A]).T, c=plt.cm.viridis(norm([VA, VB, VC, VA])), s=75, alpha=1)

n_p = 500
points     = np.zeros([n_p, 3])
intersects = np.zeros([n_p, 4])

for i in range(n_p):
    p_random = [np.nan, np.nan, np.nan]
    while not diskwarp.fmodule.pointintriangle(p_random[:2], A[:2], B[:2], C[:2]):
        p_random = vmin + np.random.rand(3) * (vmax - vmin)
        
    #intersect = intersect_tri(A, B, C, p_random)
    intersect = diskwarp.fmodule.intersect_triangle(A, B, C, p_random)
    value = diskwarp.fmodule.barycenter_interpolate(A, B, C, intersect, VA, VB, VC)
    
    points[i, :3] = p_random
    intersects[i, 0:3] = intersect
    intersects[i, 3] = value

ax.scatter(*intersects[:, :3].T, c=plt.cm.viridis(norm(intersects[:, 3])), s=10, alpha=1)
ax.set_zlabel('z')